In [1]:
import pandas as pd 
import numpy as np 
import math 
import random 
from datetime import datetime
from time import time, ctime

In [2]:
#               0  1   2   3   4   5   6   7   8   9  10     ----- matriz com as distancias entre as US
matriz = [    [ 0, 31, 17, 13, 13, 23, 8, 15, 15, 14, 31],  #0
              [ 28, 0, 18, 33, 27, 16, 27, 17, 24, 30, 15], #1
              [ 16, 18, 0, 23, 14, 26, 22, 14, 26, 27, 28], #2
              [ 14, 31, 23, 0, 15, 32, 13, 19, 23, 15, 34], #3
              [ 11, 25, 14, 13, 0, 27, 16, 14, 23, 22, 29], #4
              [ 21, 19, 27, 29, 27, 0, 20, 16, 9, 21, 16],  #5
              [ 8, 28, 20, 12, 16, 22, 0, 16, 15, 8, 30],   #6
              [ 15, 15, 13, 20, 15, 15, 16, 0, 12, 19, 19], #7
              [ 14, 20, 24, 22, 20, 9, 14, 13, 0, 16, 23],  #8
              [ 14, 28, 24, 13, 21, 20, 8, 18, 15, 0, 29],  #9
              [ 28, 15, 26, 33, 27, 16, 29, 19, 24, 31, 0]] #10


# --- Modificador de tempo relacionado à região de cada US
hrdepico = [1.465, 1.55, 1.35, 1.406, 1.7, 1.676, 1.475, 1.643, 1.517, 1.384, 1.559]

In [3]:
# Classe para lidar com horario

class hour:
    def __init__(self, hora, minuto):
        self.hora = hora
        self.minuto = minuto

In [4]:
def soma(hora1, hora2):
    hora = hour
    hora.hora = hora1.hora + hora2.hora
    hora.minuto = hora1.minuto + hora2.minuto
    if hora.minuto > 59:
        hora.minuto = hora.minuto - 60
        hora.hora += 1
    return hora

In [5]:
# Pega a distancia de uma US para a outra 

def distancia_us(a, b, matriz, modificador):
    return (matriz[a][b])*modificador

In [6]:
# obtem o vetor com a ordem em que as US devem ser visitadas

def obter_caminho(tamanho, redo):
    permutacao = []
    indices = [i for i in range(11)]
    permutacao.append(0)
    for i in range(1, tamanho):
        aleatoria = random.randint(1, len(indices)-1)
        permutacao.append(indices[aleatoria])
        indices.remove(indices[aleatoria])
    permutacao.append(0)
    return permutacao

In [7]:
# Obter o tempo que o trajeto total vai demorar 

def obter_fitness(permutacao, matriz, hora, hrdepico):
    fit = 0    
    for i in range(len(permutacao)-1):
        if ((hora.hora >= 7 and hora.hora <= 10) or (hora.hora >= 17 and hora.hora <=  20)):
            modificador = hrdepico[i]
        else: modificador = 1
        fit = fit + distancia_us(permutacao[i], permutacao[i+1], matriz, modificador)
    return fit

In [8]:
#Seleciona n individuos aleatorios da populacao

def seleciona(populacao, n):
    selecao = []
    pop = populacao.copy()
    for i in range(0,n):
        aleatorio = random.randint(0, len(pop)-1)
        selecao.append(pop[aleatorio])
        pop.remove(pop[aleatorio])
    return selecao

In [9]:
# Combina o "pedaco" dos pais para formar o filho e evitar a repeticao de US no vetor 

def combina(cross1, cross2, p):
    filho1 = cross1.copy()
    filho2 = cross2.copy()

    for i in range(0, p):
        filho1[i] = cross2[i]
    
    j = 0 

    for i in range(p, len(cross2)):
        if filho1[i] in filho1[0:p:]:
            for w in range(j, len(cross1)):
                if not (cross1[w] in filho1):
                    filho1[i] = cross1[w]
                    break
            j = j + 1

    for i in range(0, p):
        filho2[i] = cross1[i]

    j = 0
    for i in range(p, len(cross1)):
        if filho2[i] in filho2[0:p:]:
            for w in range(j, len(cross2)):
                if not (cross2[w] in filho2):
                    filho2[i] = cross2[w]
                    break
            j = j + 1

    return filho1, filho2

In [10]:
#mutacao aleatoria nos filhos 1 e 2 - 1% de chance 

def mutation(filho1, filho2):
    mutated1 = filho1.copy()
    mutated2 = filho2,copy()
    
    if 2 == random.randint(0, 100):
        aux = mutated1[len(mutated1)-1]
        mutated1[len(mutated1)-1] = mutated1[len(mutated1)-2]
        mutated1[len(mutated1)-2] = aux

    if 2 == random.randint(0, 100):
        aux = mutated2[len(mutated2)-1]
        mutated2[len(mutated2)-1] = mutated2[len(mutated2)-2]
        mutated2[len(mutated2)-2] = aux

    return mutated1, mutated2

In [11]:
# Crossover OX modificado

def crossover(selecao, matriz):
    selecao_copy = selecao.copy()
    recombinacoes = []
    for i in range(len(selecao_copy)//2):
        c1 = random.randint(0, len(selecao_copy)-1)
        cross1 = selecao_copy[c1]['permutacao']
        selecao_copy.remove(selecao_copy[c1])

        c2 = random.randint(0, len(selecao_copy)-1)
        cross2 = selecao_copy[c2]['permutacao']
        selecao_copy.remove(selecao_copy[c2])

        tamanho_corte = random.randint(1, len(cross1)-1)
        filho1, filho2 = combina(cross1, cross2, tamanho_corte)

        fitness1 = obter_fitness(filho1, matriz, hora, hrdepico)
        fitness2 = obter_fitness(filho2, matriz, hora, hrdepico)

        recombinacoes.append({'fitness' : fitness1, 'permutacao' : filho1})
        recombinacoes.append({'fitness' : fitness2, 'permutacao' : filho2})

    if len(selecao_copy) % 2 != 0:
        recombinacoes.append(selecao[random.randint(0, len(selecao) - 1)])

    return recombinacoes

In [79]:
# Atualiza o vetor de US a serem visitadas levando em consideração o modificador de hrdepico()
# O horário de pico está definido como sendo da 7:00 ás 10:00 e das 17:00 ás 20:00 com base no Índice de Tempo de Viagem 

def prox_us(melhor_individuo, hora):

    melhor_individuo['permutacao'].remove(melhor_individuo['permutacao'][0])
    melhor_individuo['fitness'] = obter_fitness(melhor_individuo['permutacao'], matriz, hora, hrdepico)
    if ((hora.hora >= 7 and hora.hora <= 10) or (hora.hora >= 17 and hora.hora <=  20)):
            modificador = hrdepico[melhor_individuo['permutacao'][0]]
    else: modificador = 1
    if len(melhor_individuo['permutacao'])>1:
        dist = ((matriz[melhor_individuo['permutacao'][0]][melhor_individuo['permutacao'][1]])*modificador)//1
        return melhor_individuo, dist

    else: return "Retornou à Rodoviária"

# Main

In [86]:
# Informacoes para a rota

hora = hour
geracoes = 10000
individuos = 1000
convergencia = 0.95
hora.hora = 14
hora.minuto = 0

In [107]:
%%time 
#Calcula a rota inicial e informa o matorista 

populacao = [{'permutacao' : obter_caminho(11, 0), 'fitness' : 0} for i in range(individuos)]

for individuo in populacao:
    individuo['fitness'] = obter_fitness(individuo['permutacao'], matriz, hora, hrdepico)

for i in range(0, geracoes):
    selecionados = seleciona(populacao, individuos//2)

    recombinacoes = crossover(populacao, matriz)

    nova_populacao = selecionados + recombinacoes

    nova_populacao.sort(key = lambda x : x['fitness'])

    populacao = nova_populacao[0 : individuos : ]

    fitnessess = [d['fitness'] for d in populacao]

    if(len([individuo for individuo in populacao if individuo['fitness'] == min(fitnessess)])/individuos > convergencia):
        break

melhor_individuo = populacao[0]

print(melhor_individuo)

{'fitness': 215.34799999999998, 'permutacao': [0, 6, 9, 3, 4, 2, 7, 1, 10, 5, 8, 0]}
Wall time: 2.34 s


In [105]:
# Informacoes para a rota

hora = hour
hora.hora = 18
hora.minuto = 0
iter_pico = 0
us_visitada = []

In [106]:
# Proxima unidade a ser visita e se for Hr de pico ele recalcula a rota, se a nova rota tiver fitness melhor que a primeira, ele substitui e informa o motorista 

if ((hora.hora >= 7 and hora.hora <= 10 and iter_pico == 0) or (hora.hora >= 17 and hora.hora <=  20 and iter_pico == 0)):
    populacao = [{'permutacao' : obter_caminho(11, 1), 'fitness' : 0} for i in range(individuos)]
    for x in range(len(us_visitada)):
        populacao['permutacao'].remove(us_visitada[x])
        populacao['permutacao'] = melhor_individuo['permutacao'][0] + populacao['permutacao']
        populacao['permutacao'].remove(melhor_individuo['permutacao'][0])
    for individuo in populacao:
        individuo['fitness'] = obter_fitness(individuo['permutacao'], matriz, hora, hrdepico)

    for i in range(0, geracoes):
        selecionados = seleciona(populacao, individuos//2)

        recombinacoes = crossover(populacao, matriz)

        nova_populacao = selecionados + recombinacoes

        nova_populacao.sort(key = lambda x : x['fitness'])

        populacao = nova_populacao[0 : individuos : ]

        fitnessess = [d['fitness'] for d in populacao]


        if(len([individuo for individuo in populacao if individuo['fitness'] == min(fitnessess)])/individuos > convergencia):
            break

    iter_pico = 1
    modificador = hrdepico[melhor_individuo['permutacao'][0]]
    novo_individuo = populacao[0]
    if novo_individuo['fitness'] < melhor_individuo['fitness']:
        melhor_individuo = novo_individuo
    print('Rota recalculada:',melhor_individuo['permutacao'],'Proxima unidade em:',(matriz[melhor_individuo['permutacao'][0]][melhor_individuo['permutacao'][1]]*modificador)//1,'minutos')

elif len(melhor_individuo['permutacao'])>2:
    us_visitada.append(melhor_individuo['permutacao'][0])
    indv, dist = prox_us(melhor_individuo, hora)
    print(indv, "Proxima unidade em:",dist,"minutos")
else: print(prox_us(melhor_individuo, hora))

Rota recalculada: [0, 6, 9, 3, 4, 2, 7, 1, 10, 5, 8, 0] Proxima unidade em: 11.0 minutos


4.0